In [1]:
# import the neccessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle

In [2]:
# load the preprocessed data
data = pd.read_csv('../data/data_modelling.csv', index_col=0)

In [3]:
# check shape
data.shape

(894744, 26)

In [4]:
X = data.drop(['loan_age'], axis=1)
y = data['loan_age']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# define the validation data
X_train_val, X_val, y_train_val, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [6]:
print(f'training set row: {X_train_val.shape[0]}',
      f'validation set row: {X_val.shape[0]}',
      f'test set row: {X_test.shape[0]}',)

training set row: 536846 validation set row: 178949 test set row: 178949


In [7]:
# standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# fit and tranform
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# put in a DataFrame
X_train_scaled = pd.DataFrame(X_train, columns=X.columns)
X_val_scaled = pd.DataFrame(X_val, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test, columns=X.columns)

In [8]:
#check shape of X_train
X_train_scaled.shape

(715795, 25)

In [9]:
# check shape of X_test
X_test_scaled.shape

(178949, 25)

In [10]:
# fit the regression model
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# sklearn regression
reg = LinearRegression().fit(X_train_scaled, y_train)
print(reg.score(X_train_scaled, y_train))
print(reg.coef_)
print(reg.intercept_)

0.6464411842919604
[-4.90461123e+00 -4.08143663e-01 -6.97476224e+00 -4.35941346e-01
 -2.04917945e-02  1.84274395e+01 -9.67686720e+00  1.87149583e+00
  4.16891728e-02  7.46618428e-02  1.07226635e+00  1.89709030e+00
  2.36027539e-01  1.48947926e-01  6.83767816e-02  1.11126547e-03
 -1.27130595e-01 -1.70080668e-02 -6.69439340e-02 -3.94195208e-02
 -1.11329206e-01 -6.80876957e-03  6.07438463e-02 -1.05941594e-01
  1.63409397e-01]
22.44653579312905


In [11]:
# reset y index for statsmodel
y_train_reset = y_train.reset_index(drop=True)
y_test_reset = y_test.reset_index(drop=True)
y_val_reset = y_val.reset_index(drop=True)

In [12]:
# statmodel regression
X_train_scaled = sm.add_constant(X_train_scaled) # adding a constant
lin_reg = sm.OLS(y_train_reset, X_train_scaled)
model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:               loan_age   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.646
Method:                 Least Squares   F-statistic:                 5.235e+04
Date:                Fri, 28 Mar 2025   Prob (F-statistic):               0.00
Time:                        14:34:35   Log-Likelihood:            -2.4567e+06
No. Observations:              715795   AIC:                         4.913e+06
Df Residuals:                  715769   BIC:                         4.914e+06
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [13]:
X_test_scaled = sm.add_constant(X_test_scaled) # adding a constant

In [14]:
from sklearn.metrics import r2_score 

r2_score(y_test, model.predict(X_test_scaled))

0.6463760720772995

In [15]:
# fit the XGBoost
grad = xgb.XGBRegressor().fit(X_train_scaled, y_train)
grad

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

# Model Evaluation

In [16]:
lin_predict = model.predict(X_test_scaled)
grad_pred = grad.predict(X_test_scaled)
metrics = {}

predictions = {'Linear Regression': lin_predict,
               'Gradient Boosting': grad_pred}

n = len(y_test)

# Compute metrics
for model_name, y_pred in predictions.items():
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    p = X_test.shape[1]
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    
    metrics[model_name] = {'MSE': mse,
                           'RMSE': rmse,
                           'MAE': mae,
                           'R-squared': r2,
                           'Adjusted R-squared': adj_r2}
metrics_df = pd.DataFrame(metrics).map(lambda x: "{:.2e}".format(x))
metrics_df

,Linear Regression,Gradient Boosting
MSE,5.60e+01,7.34e+00
RMSE,7.48e+00,2.71e+00
MAE,5.53e+00,1.46e+00
R-squared,6.46e-01,9.54e-01
Adjusted R-squared,6.46e-01,9.54e-01


In terms of evaluating criteria:

RMSE, MSE, 
 and Adj 
 are all linked to the squared error. RMSE has the benefit of being interpretable in terms of actual units, and 
 gives a good relative measure of success.

MAE is linked to the observed error, not the model's loss function (squared error).

Overall, the strongest selectors for model fit are RMSE, MSE, 
 and Adj 
 - these are all linked to the actual squared error and therefore give the best indication of model fit.

MAE is suitable as a reporting metric to stakeholders, but isn't suitable for model selection because it is only indirectly linked to goodness of fit.

Feature Selection
Future goal to explore methods such as RFECV or Forward/Backward selection to reduce the model's dimensionality

# Hyperparameter Tuning

In [17]:
#=========================================================================
# XGBoost regression: 
# Parameters: 
# n_estimators  "Number of gradient boosted trees. Equivalent to number 
#                of boosting rounds."
# learning_rate "Boosting learning rate (also known as “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make 
#                the model more complex and more likely to overfit." 
#=========================================================================
## put code here
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error

regressor=xgb.XGBRegressor(eval_metric='rmsle')

#=========================================================================
# exhaustively search for the optimal hyperparameters
#=========================================================================
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# set up our search grid


In [18]:

    # Learning rate (eta)
param_grid = {"learning_rate": [0.001, 0.01, 0.1],
    
    # Maximum depth of a tree
    "max_depth": [3, 5, 7, 10],
    
    # Minimum sum of instance weight (hessian) needed in a child
    "min_child_weight": [0.1, 1, 10.0],
    
    # Subsample ratio of training instance
    "subsample": [0.1, 0.5, 1.0],
    
    # Subsample ratio of columns when constructing each tree
    "colsample_bytree": [0.5, 0.7, 1.0],
    
    # Subsample ratio of columns for each split
    "colsample_bylevel": [0.5, 0.7, 1.0],
    
    # Subsample ratio of columns for each node
    "colsample_bynode": [0.5, 0.7, 1.0],
    
    # L1 regularization term on weights
    "reg_alpha": [0.01, 0.1, 10.0],
    
    # L2 regularization term on weights
    "reg_lambda": [0.01, 0.1, 10.0],
    
    # Minimum loss reduction required to make a further partition on a leaf node
    "gamma": [0.01, 1, 10.0],
    
    # # Base score for the initial predictions
    "base_score": [0.1, 0.5, 0.7, 1.0],

    "max_delta_step": [0, 1, 5, 10, 20]}

In [19]:
import multiprocessing

multiprocessing.cpu_count()

12

In [20]:
# Initialize XGBoost regressor
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# Perform Grid Search
rand_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid, cv=5, scoring="neg_mean_absolute_error", verbose=5, n_jobs=8, n_iter=20)
rand_search.fit(X_train_scaled, y_train)

# Best parameters & best score
print("Best hyperparameters:", rand_search.best_params_)
print("Best Score (Negative MAE):", rand_search.best_score_)



Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best hyperparameters: {'subsample': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 10.0, 'min_child_weight': 10.0, 'max_depth': 10, 'max_delta_step': 10, 'learning_rate': 0.1, 'gamma': 1, 'colsample_bytree': 1.0, 'colsample_bynode': 1.0, 'colsample_bylevel': 0.7, 'base_score': 0.1}
Best Score (Negative MAE): -1.3774032335871969


In [21]:
# Use the best hyperparameters from Random Search
best_params = {'subsample': 0.5, 
               'reg_lambda': 0.1, 
               'reg_alpha': 0.1, 
               'min_child_weight': 1, 
               'max_depth': 10, 
               'max_delta_step': 20, 
               'learning_rate': 0.1, 
               'gamma': 0.01, 
               'colsample_bytree': 0.5, 
               'colsample_bynode': 1.0, 
               'colsample_bylevel': 0.7, 
               'base_score': 0.7}

# Initialize & train XGBoost Regressor
best_xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42, **best_params).fit(X_train_scaled, y_train)

In [22]:
# Predict on validation set
y_pred = best_xgb_model.predict(X_test_scaled)

In [23]:
with open('../models/tuned_model.pkl', 'wb') as f:
    pickle.dump(best_xgb_model, f)